In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import sys
sys.path.append("C:\\Users\\Sam Yu\\Documents\\GitHub\\LiquidCrystals")

from postprocessing import LCSystem

In [ ]:
datasets_path = "C:\\Users\\Sam Yu\\Desktop\\School\\4A\\Phys_437A_Research_Project\\datasets\\r=14\\"

In [ ]:
lc_systems_wrt_density = dict()

for _path_ in os.listdir(datasets_path):
    
    full_path = os.path.join(datasets_path, _path_, 'instanceRun')
    
    if os.path.exists(os.path.join(full_path, "MonteCarlo_Annulus_SimNotes.txt")):
        lc = LCSystem(lc_data_path=full_path)
        global_packing_fraction = lc.sim_params['reduced density']
        
        lc_systems_wrt_density[global_packing_fraction] = lc
        
print(lc_systems_wrt_density.keys())

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
number_of_features = 10
datapoints_per_step = 5

# number of rows AKA number of data-points
# = number of densities * number of captures per density >= 1e6 * number of data-points per capture 
 
number_of_densities = len(lc_systems_wrt_density)
number_of_captures = len([step for step in
                          lc_systems_wrt_density[0.2913752913752914].system_state_at_step
                          if step >= 1e6])

n_rows = number_of_densities * number_of_captures * datapoints_per_step
n_cols = number_of_features

data_matrix = np.zeros((n_rows, n_cols))

for density in lc_systems_wrt_density.keys():
    
    system_state_at_step = lc_system_wrt_density[density]
    
    for mc_step in system_state_at_step:
        
        if mc_step >= 1e6:

In [ ]:
import math
from numpy.linalg import norm

def create_feature_vectors_from_snapshot(coordinates, num_features):
    
    assert (num_features < len(coordinates)), \
    f"Number of features {num_features} cannot be greater than number of particles {len(coordinates)}"
    
    #print(f"Input coordinates: {coordinates}")
    
    N = len(coordinates)
    nn_sampling_number = math.floor(N / num_features)
    print(f"Nearest neighbor sampling number: {nn_sampling_number}")
        
    # set a random seed for reproducibility
    rng = np.random.default_rng(666)
    probe_indices = rng.choice(N, size=num_features, replace=False)
    
    # x, y positions for particle center of masses
    spatial = [c[:2] for c in coordinates]
    # angle of long axis with respect to x for particles
    angular = [(c[-1] % np.pi) for c in coordinates]
    
    feature_vectors = []
    
    for probe_index in probe_indices:
        
        # list of distances relative to probe particle
        chosen_coord = coordinates[probe_index]
        
        norm2 = lambda x, y: (np.array(x) - np.array(y)) @ (np.array(x) - np.array(y))
        dist_to_P = lambda x: norm2(x[:2], chosen_coord[:2])
        
        # may abstract as function param later
        feature_func = lambda x, y: abs(np.cos(x - y))
        
        nn_sorted = sorted(coordinates, key=dist_to_P)
        #print(f"Nearest neighbor sort: {nn_sorted}")
        
        # feature vector
        fv = []
        
        # add feature based on nearest neighbor distance
        for i, c in enumerate(nn_sorted):
            
            if (i > 0) and (i % nn_sampling_number) == 0:
                
                feature = feature_func(c[-1], chosen_coord[-1])
                fv.append(feature)
            
            # Break the addition of features if total number of features is met
            if len(fv) == num_features:
                break
                
        feature_vectors.append(fv)
    
    return feature_vectors

In [ ]:
create_feature_vectors_from_snapshot([(1, 2), (3, 4), (5, 6), (9, 10), (5, 8), (7, 8), (9, 11)], 2)

In [ ]:
test_lc_system = lc_systems_wrt_density[0.2913752913752914]

print(test_lc_system.sim_params)

In [ ]:
test_coords = test_lc_system.system_state_at_step[19999]
create_feature_vectors_from_snapshot(test_coords, 10)